# sklearn IRIS
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=benchmark&ea=bentoml-benchmark&dt=sklearn-iris-bentoml)

Before we start

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
import bentoml
NAME = 'sklearn_iris'

# load data & train model

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

    
# add parameters for tuning
num_estimators = 100

# train the model
model = RandomForestRegressor(n_estimators=num_estimators)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('predictions: ', predictions)

# log model performance 
mse = mean_squared_error(y_test, predictions)
print("  mse: %f" % mse)

predictions:  [2.   1.   0.   1.   1.62 0.   1.23 1.   0.   1.   2.   1.   0.   2.
 0.   1.88 2.   2.   0.   0.   1.   2.   1.   1.33 1.52 1.88 1.   1.
 2.   2.  ]
  mse: 0.088847


In [3]:
%%writefile {NAME}.py

import bentoml
from bentoml.artifact import SklearnModelArtifact
from bentoml.handlers import DataframeHandler


@bentoml.env()
@bentoml.artifacts([SklearnModelArtifact('model')])
class BentoSvc(bentoml.BentoService):
    @bentoml.api(DataframeHandler)
    def predict(self, inputs):
        outputs = self.artifacts.model.predict(inputs.to_numpy())
        return outputs

Overwriting sklearn_iris.py


In [4]:
from sklearn_iris import BentoSvc

bento_svc = BentoSvc()
bento_svc.pack("model", model)
saved_path = bento_svc.save()
print(saved_path)

[2020-05-09 04:02:16,814] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-09 04:02:17,084] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.4.9+222.g54dcd0e.dirty
creating BentoML-0.4.9+222.g54dcd0e.dirty/BentoML.egg-info
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/artifact
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/bundler
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/cli
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/clipper
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/configuration
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/configuration/__pycache__
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/deployment
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/deployment/aws_lambda
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/deployment/sagemaker
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/handlers
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/marshal
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/migrations
creating BentoML-0.4.9+222.g54d

In [5]:
saved_path

'/home/ec2-user/bentoml/repository/BentoSvc/20200509040217_9E5C59'

# Build & Run Bento Service in Docker

In [6]:
!cd {saved_path}
IMG_NAME = saved_path.split('/')[-1].lower()
!docker build -t {IMG_NAME} {saved_path}

# Optional: set your prefer PYPI mirror
# --build-arg PIP_TRUSTED_HOST=192.168.138.2 \
# --build-arg PIP_INDEX_URL=http://192.168.138.2/simple \

Sending build context to Docker daemon  592.4kB
Step 1/18 : FROM continuumio/miniconda3:4.8.2
 ---> b4adc22212f1
Step 2/18 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 53fb2c825bc1
Step 3/18 : RUN conda install pip numpy scipy       && pip install gunicorn
 ---> Using cache
 ---> b599776a4924
Step 4/18 : COPY . /bento
 ---> b2fa64293d1a
Step 5/18 : WORKDIR /bento
 ---> Running in f1d8ec7b6453
Removing intermediate container f1d8ec7b6453
 ---> 5ca90779ed9a
Step 6/18 : RUN if [ -f /bento/setup.sh ]; then /bin/bash -c /bento/setup.sh; fi
 ---> Running in 97991dc69c78
Removing intermediate container 97991dc69c78
 ---> e56172dd754f
Step 7/18 : RUN conda env update -n base -f /bento/environment.yml
 ---> Running in 1bad1a9a8bca
Solving environment: ...working... done

urllib3-1.25.8       | 169 KB    | ########## | 100% 
scipy-1.2.1          | 

In [17]:
from bentoml.utils import detect_free_port
PORT = detect_free_port()
print(PORT)
!docker run -itd -p {PORT}:5000 --cpus 1 -e FLAGS="--workers 1 --enable-microbatch" {IMG_NAME}:latest
server_url = f"http://127.0.0.1:{PORT}"
print(server_url)

56011
89aa2754532e43f5c82f97e38e0673d50ecc7d7a8e9f0bdef77701785926a18b
http://127.0.0.1:56011


# Test with requests

In [5]:
import json
import requests
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

headers = {"content-type": "application/json"}
raw_data = X_test
data = pd.DataFrame(raw_data).to_json()

json_response = requests.post(f'{server_url}/predict',
                              data=data, headers=headers)
print(json_response)
print(json_response.json())

<Response [200]>
[2.0, 1.0, 0.0, 1.0, 1.54, 0.0, 1.25, 1.0, 0.0, 1.0, 2.0, 1.0, 0.0, 2.0, 0.0, 1.87, 2.0, 2.0, 0.0, 0.0, 1.0, 2.0, 1.0, 1.38, 1.55, 1.87, 1.0, 1.0, 2.0, 2.0]


# Benchmark

In [4]:
import pandas as pd
import json
import copy
import random


def get_request_producer():

    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    iris = datasets.load_iris()
    x = iris.data[:, 2:]
    y = iris.target

    url = server_url + '/predict'
    method = "POST"
    headers = {"content-type": "application/json"}
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=7)
    base_data = pd.DataFrame(X_test[:1]).to_dict()

    def _gen_data():
        raw = copy.deepcopy(base_data)
        raw[0][0] += random.random() / 10000
        raw[1][0] += random.random() / 10000
        data = json.dumps(raw)
        return url, method, headers, data

    return _gen_data

# test
get_request_producer()()

('http://127.0.0.1:53093/predict',
 'POST',
 {'content-type': 'application/json'},
 '{"0": {"0": 5.100024923783534}, "1": {"0": 1.8000719264537908}}')

In [6]:
from bentoml.utils.benchmark import BenchmarkClient
b = BenchmarkClient(get_request_producer(), lambda: 1, timeout=0.32)
b.start_session(60, 900, 600)

======= Session started! =======

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │       0 │        0 │             nan │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │        0 │             nan │                   │
╘══════════╧═════════╧══════════╧═════════════════╧═══════════════════╛

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │     968 │    483.5 │       0.0383909 │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │      48 │     24   │       0.0365302 │                   │
╘══════════╧═════════╧═══════